In [1]:
# Block 1: imports & seeds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import copy
from typing import List, Tuple, Dict, Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch_geometric.data import Data
from torch_geometric.nn import GATConv

from sklearn.metrics import roc_auc_score, average_precision_score
from statsmodels.tsa.stattools import grangercausalitytests

# reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

c:\Users\chand\anaconda3\envs\AllPackages\Lib\site-packages\torch_geometric\typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
c:\Users\chand\anaconda3\envs\AllPackages\Lib\site-packages\torch_geometric\typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
# Block 2: create synthetic minute-level data (single person)
def generate_synthetic_minute_data(days=7, signals=None, anomaly_spec=None, seed=SEED):
    """
    Generates minute-by-minute multivariate signals for `days`.
    signals: dict of base level and circadian influence multipliers, e.g.
      {"sleep_quality": (0.8, -0.2), "stress": (0.2, 0.15), ...}
    anomaly_spec: list of dicts describing injected anomalies:
      [{"type":"sleep_deprivation","start_min":1234,"duration_min": 60*48, "magnitude": -0.6}, ...]
    Returns: df (DatetimeIndex), anomaly_label (binary array)
    """
    rng = np.random.RandomState(seed)
    minutes_per_day = 24 * 60
    total_minutes = days * minutes_per_day
    t = np.arange(total_minutes)
    time_index = pd.date_range("2025-01-01", periods=total_minutes, freq="1min")

    # default signals if none provided
    if signals is None:
        signals = {
            "sleep_quality": (0.7, -0.25),
            "stress_level": (0.25, 0.2),
            "cardio_stability": (0.85, -0.05),
            "pulmonary_efficiency": (0.8, 0.03),
            "cognitive_load": (0.45, 0.18)
        }

    circadian = np.sin(2 * np.pi * t / minutes_per_day)  # daily periodicity

    data = {}
    for name, (base, circ_amp) in signals.items():
        # base + circ_amp * circadian + small random walk + Gaussian noise
        rw = np.cumsum(rng.normal(0, 0.0005, size=total_minutes))  # slow drift
        signal = base + circ_amp * circadian + rw + rng.normal(0, 0.03, size=total_minutes)
        data[name] = signal

    anomaly_label = np.zeros(total_minutes, dtype=int)

    # default anomaly_spec if none provided (simulate a 48h sleep deprivation and other events)
    if anomaly_spec is None:
        anomaly_spec = []
        # sleep deprivation starting day 2 at 00:00 for 48 hours
        sd_start = minutes_per_day * 1
        anomaly_spec.append({"type":"sleep_deprivation", "start_min": sd_start, "duration_min": 60*48, "magn": -0.6})
        # stress spikes: random short spikes
        for i in range(6):
            s = rng.randint(0, total_minutes-30)
            anomaly_spec.append({"type":"stress_spike", "start_min": s, "duration_min": rng.randint(10,60), "magn": 0.6})
        # pulmonary episodes
        for i in range(3):
            s = rng.randint(0, total_minutes-120)
            anomaly_spec.append({"type":"pulmonary_episode","start_min": s, "duration_min": rng.randint(30,180), "magn": -0.4})

    # apply anomalies
    for ev in anomaly_spec:
        s = ev["start_min"]
        e = min(s + ev["duration_min"], total_minutes)
        typ = ev["type"]
        magn = ev.get("magn", -0.4)
        if typ == "sleep_deprivation":
            # lower sleep_quality; increase stress gradually and reduce cardio after a lag
            data["sleep_quality"][s:e] += magn
            # stress increases during and after
            data["stress_level"][s:e + 60*12] += 0.3  # some carryover 12 hours
            # cardio affected after a delay of 6 hours
            delay = 60*6
            data["cardio_stability"][s+delay:e+delay] += -0.15
            anomaly_label[s:e] = 1
        elif typ == "stress_spike":
            data["stress_level"][s:e] += magn
            data["cognitive_load"][s:e] += 0.2
            anomaly_label[s:e] = 1
        elif typ == "pulmonary_episode":
            data["pulmonary_efficiency"][s:e] += magn
            anomaly_label[s:e] = 1

    # clip to 0-1 meaningful range
    for k in data:
        data[k] = np.clip(data[k], 0.0, 1.0)

    df = pd.DataFrame(data, index=time_index)
    return df, anomaly_label

# quick test
df, anomaly_label = generate_synthetic_minute_data(days=7)
print(df.shape, anomaly_label.sum(), "anomaly minutes")
df.head()


(10080, 5) 3192 anomaly minutes


,sleep_quality,stress_level,cardio_stability,pulmonary_efficiency,cognitive_load
2025-01-01 00:00:00,0.755708,0.205395,0.830908,0.767178,0.510991
2025-01-01 00:01:00,0.659859,0.246476,0.844178,0.792851,0.417215
2025-01-01 00:02:00,0.706531,0.238954,0.868644,0.811848,0.416396
2025-01-01 00:03:00,0.744998,0.281297,0.838533,0.839644,0.434464
2025-01-01 00:04:00,0.714757,0.273863,0.833889,0.812706,0.471335


In [3]:
# Block 3: sliding window creation + directed edge construction functions
from scipy.signal import correlate

def sliding_windows(df: pd.DataFrame, window_minutes=60*24, stride_minutes=60*6):
    """
    Create overlapping windows. Returns list of (node_time_matrix, start_idx)
    node_time_matrix shape: (num_nodes, window_minutes)
    """
    windows = []
    start_idxs = []
    n = len(df)
    for start in range(0, n - window_minutes - 1, stride_minutes):
        w = df.iloc[start:start+window_minutes].values.T.astype(np.float32)
        windows.append(w)
        start_idxs.append(start)
    return windows, start_idxs

def directed_edges_by_lag(node_ts: np.ndarray, max_lag:int=60, corr_threshold:float=0.25):
    """
    node_ts: (num_nodes, window_size)
    returns list of (src, dst, weight, lag) meaning src leads dst by `lag` minutes
    Uses normalized cross-correlation; looks for positive lags where src leads dst.
    """
    num_nodes, L = node_ts.shape
    edges = []
    for i in range(num_nodes):
        xi = node_ts[i] - node_ts[i].mean()
        sxi = xi.std() if xi.std() != 0 else 1e-6
        for j in range(num_nodes):
            if i == j: continue
            xj = node_ts[j] - node_ts[j].mean()
            sxj = xj.std() if xj.std() != 0 else 1e-6
            # full cross-correlation
            xc = correlate(xj, xi, mode='full')  # correlation of xi -> xj: positive lag means xi leads xj
            lags = np.arange(-L+1, L)
            mid = len(xc)//2
            # we want positive lags (xi leads xj) up to max_lag
            pos_region = xc[mid+1: mid+1+max_lag]
            if pos_region.size == 0:
                continue
            best_idx = np.argmax(np.abs(pos_region))
            best_val = pos_region[best_idx] / (L * sxi * sxj)
            lag = best_idx + 1
            if abs(best_val) >= corr_threshold:
                edges.append((i, j, float(best_val), int(lag)))
    return edges

def granger_directed_edges(node_ts: np.ndarray, maxlag:int=10, p_threshold:float=0.05):
    """
    Pairwise Granger causality on the window. node_ts shape (num_nodes, L).
    Returns list of (src, dst, pval, best_lag) where src Granger-causes dst.
    NOTE: this is heavier; use smaller maxlag if window is short.
    """
    num_nodes, L = node_ts.shape
    edges = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i==j: continue
            # statsmodels expects shape (T, 2) as [target, cause]
            data = np.vstack([node_ts[j], node_ts[i]]).T  # test whether i causes j
            try:
                res = grangercausalitytests(data, maxlag=maxlag, verbose=False)
                # collect min pval across lags for ssr_ftest
                pvals = [(lag, res[lag][0]['ssr_ftest'][1]) for lag in res]
                best_lag, best_p = min(pvals, key=lambda x: x[1])
                if best_p < p_threshold:
                    edges.append((i, j, float(best_p), int(best_lag)))
            except Exception:
                # numerical issues sometimes; skip
                continue
    return edges


In [4]:
# Block 4: create PyG Data objects; combine cross-corr & granger info into edge_attr
def window_to_pyg(node_ts: np.ndarray, node_names: List[str],
                  use_granger=False, corr_thresh=0.25, max_lag=60):
    """
    node_ts: (num_nodes, window_size)
    returns torch_geometric.data.Data with x (per-node features) and directed edge_index (2 x E)
    and edge_attr tensor (E x 3) for (corr_weight, lag, granger_p) where granger_p may be np.nan
    """
    num_nodes = node_ts.shape[0]
    # summary node features (mean,std,last,slope)
    means = node_ts.mean(axis=1)
    stds = node_ts.std(axis=1)
    last = node_ts[:, -1]
    slopes = np.array([np.polyfit(np.arange(node_ts.shape[1]), node_ts[i], 1)[0] for i in range(num_nodes)])
    x = np.stack([means, stds, last, slopes], axis=1)
    # directed cross-correlation edges
    corr_edges = directed_edges_by_lag(node_ts, max_lag=max_lag, corr_threshold=corr_thresh)
    # optional granger
    gr_edges = {}
    if use_granger:
        g = granger_directed_edges(node_ts, maxlag=min(10, node_ts.shape[1]//10), p_threshold=0.05)
        for (src, dst, pval, lag) in g:
            gr_edges[(src,dst)] = (pval, lag)
    # build edge lists and attributes
    if len(corr_edges) == 0:
        # fallback to small chain if no directed corr
        corr_edges = [(i, i+1, 0.1, 1) for i in range(num_nodes-1)]

    edge_index = np.array([[e[0] for e in corr_edges], [e[1] for e in corr_edges]])
    edge_attrs = []
    for (src, dst, weight, lag) in corr_edges:
        gr_p, gr_l = (np.nan, -1)
        if (src,dst) in gr_edges:
            gr_p, gr_l = gr_edges[(src,dst)]
        edge_attrs.append([weight, float(lag), float(gr_p if not np.isnan(gr_p) else np.nan)])
    import torch
    data = Data(x=torch.tensor(x, dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype=torch.long),
                edge_attr=torch.tensor(np.array(edge_attrs), dtype=torch.float))
    # store node names for interpretability
    data.node_names = node_names
    return data

# Example: create windows and one Data
window_size = 60*24   # 1-day windows
stride = 60*6         # every 6 hours
node_names = df.columns.tolist()
windows, starts = sliding_windows(df, window_minutes=window_size, stride_minutes=stride)
print("Total windows:", len(windows))
sample_data = window_to_pyg(windows[0], node_names, use_granger=False)
print(sample_data)


Total windows: 24
Data(x=[5, 4], edge_index=[2, 20], edge_attr=[20, 3], node_names=[5])


In [5]:
# Block 5: prepare sequences of graphs and targets (per-node next-window mean)
pyg_graphs = []
targets = []
window_start_idxs = starts

for w in windows:
    pyg_graphs.append(window_to_pyg(w, node_names, use_granger=False, corr_thresh=0.2, max_lag=180))
# target = mean of next window (per-node)
targets_list = []
for i in range(len(windows)-1):
    # we'll predict next window mean for window i -> i+1
    nxt = windows[i+1].mean(axis=1)  # (num_nodes,)
    targets_list.append(nxt)
# last window has no next; drop last pyg_graph to align sizes
pyg_graphs = pyg_graphs[:-1]
n_windows = len(pyg_graphs)
targets = torch.tensor(np.stack(targets_list, axis=0), dtype=torch.float)  # (n_windows, num_nodes)
print("Prepared", n_windows, "pyg graphs; targets shape:", targets.shape)


Prepared 23 pyg graphs; targets shape: torch.Size([23, 5])


In [6]:
# Block 6: model definition (same basic idea; uses edge_index as directed)
class GATEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, heads=2, dropout=0.2):
        super().__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=heads, concat=True)
        self.conv2 = GATConv(hidden_dim*heads, out_dim, heads=1, concat=False)
        self.dropout = dropout
    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # (num_nodes, out_dim)

class TemporalGATPredictor(nn.Module):
    def __init__(self, gat_in, gat_hidden, gat_out, lstm_hidden, num_nodes, seq_len):
        super().__init__()
        self.encoder = GATEncoder(gat_in, gat_hidden, gat_out)
        self.seq_len = seq_len
        self.num_nodes = num_nodes
        self.gat_out = gat_out
        self.lstm = nn.LSTM(input_size=gat_out * num_nodes, hidden_size=lstm_hidden, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, num_nodes)
    def forward(self, batch_graphs: List[Data]):
        # encode each snapshot
        encoded = []
        for data in batch_graphs:
            z = self.encoder(data.x, data.edge_index)  # (num_nodes, gat_out)
            encoded.append(z.view(-1))
        seq = torch.stack(encoded, dim=0).unsqueeze(0)  # shape (1, seq_len, num_nodes*gat_out)
        out, _ = self.lstm(seq)
        final = out[:, -1, :]
        pred = torch.sigmoid(self.fc(final))
        return pred.squeeze(0)  # (num_nodes,)

# hyperparams
gat_in = 4
gat_hidden = 16
gat_out = 8
lstm_hidden = 64
num_nodes = len(node_names)
seq_len = 6

model = TemporalGATPredictor(gat_in, gat_hidden, gat_out, lstm_hidden, num_nodes, seq_len)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()


In [7]:
# Block 7: train / val / test split & training loop
n = n_windows
idxs = list(range(0, n - seq_len))  # valid starting indices (predict i+seq_len)
random.shuffle(idxs)
train_frac, val_frac = 0.7, 0.15
n_train = int(train_frac * len(idxs))
n_val = int(val_frac * len(idxs))
train_idx = idxs[:n_train]
val_idx = idxs[n_train:n_train+n_val]
test_idx = idxs[n_train+n_val:]

def get_sequence(i):
    # returns sequence list of Data objects of length seq_len, and target tensor
    seq_datas = [pyg_graphs[j] for j in range(i, i + seq_len)]
    target = targets[i + seq_len]  # (num_nodes,)
    return seq_datas, target

n_epochs = 25
best_val = float('inf')
best_state = None

for epoch in range(1, n_epochs+1):
    model.train()
    total_loss = 0.0
    random.shuffle(train_idx)
    for i in train_idx:
        seq_datas, target = get_sequence(i)
        optimizer.zero_grad()
        pred = model(seq_datas)  # (num_nodes,)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train = total_loss / max(1, len(train_idx))

    # validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i in val_idx:
            seq_datas, target = get_sequence(i)
            pred = model(seq_datas)
            val_loss += criterion(pred, target).item()
    val_loss = val_loss / max(1, len(val_idx))
    if val_loss < best_val:
        best_val = val_loss
        best_state = copy.deepcopy(model.state_dict())
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} train_loss={avg_train:.5f} val_loss={val_loss:.5f}")

# load best
if best_state is not None:
    model.load_state_dict(best_state)
print("Training done. Best val loss:", best_val)


Epoch 01 train_loss=0.05729 val_loss=0.03742
Epoch 05 train_loss=0.01046 val_loss=0.00258
Epoch 10 train_loss=0.00992 val_loss=0.00306
Epoch 15 train_loss=0.00977 val_loss=0.00241
Epoch 20 train_loss=0.00977 val_loss=0.00291
Epoch 25 train_loss=0.01037 val_loss=0.00388
Training done. Best val loss: 0.0012999041937291622


In [8]:
# Block 8: evaluate across test indices -> build anomaly scores per predicted window
model.eval()
scores = []
true_labels = []
window_centers = []
preds = []

with torch.no_grad():
    for i in test_idx:
        seq, target = get_sequence(i)
        pred = model(seq).cpu().numpy()
        mse = float(np.mean((pred - target.numpy())**2))
        scores.append(mse)
        preds.append(pred)
        # window index of prediction is i + seq_len
        true_labels.append(int(anomaly_label[window_start_idxs[i + seq_len]]))  # rough alignment
        window_centers.append(window_start_idxs[i + seq_len])
scores = np.array(scores)
true_labels = np.array(true_labels)

if len(np.unique(true_labels)) > 1:
    roc = roc_auc_score(true_labels, scores)
    pr = average_precision_score(true_labels, scores)
else:
    roc, pr = np.nan, np.nan
print("ROC-AUC:", roc, "PR-AUC:", pr)


ROC-AUC: 1.0 PR-AUC: 1.0


In [9]:
# Block 9: perturbation influence function to quantify downstream effect
def perturb_and_measure(model, seq_datas, node_idx, perturbation=-0.3, which_snapshot=-1):
    """
    Perturb node `node_idx` feature(s) in snapshot `which_snapshot` (default last)
    Returns delta per-node: pred_pert - pred_orig
    """
    model.eval()
    seq_p = copy.deepcopy(seq_datas)
    # perturb mean/last/slope features for the node in the chosen snapshot
    with torch.no_grad():
        pred_orig = model(seq_datas).cpu().numpy()
    # apply perturbation to all four features of that node in snapshot
    seq_p[which_snapshot].x[node_idx] = seq_p[which_snapshot].x[node_idx] + perturbation
    with torch.no_grad():
        pred_pert = model(seq_p).cpu().numpy()
    delta = pred_pert - pred_orig
    return delta, pred_orig, pred_pert

# Example: pick a detected anomaly window in test set and compute perturbation influence
# Choose first test i where score high
if len(test_idx) == 0:
    print("No test indices")
else:
    # pick worst scoring window
    worst_idx_in_test = test_idx[np.argmax(scores)]
    seq_datas, target = get_sequence(worst_idx_in_test)
    # choose node 0 (sleep_quality) perturbation to see downstream effect
    delta, orig, pert = perturb_and_measure(model, seq_datas, node_idx=0, perturbation=-0.4, which_snapshot=-1)
    print("Perturbation delta (pert - orig) per node:", {node_names[i]: float(delta[i]) for i in range(len(node_names))})
    # brief narrative
    affected = sorted([(node_names[i], abs(delta[i])) for i in range(len(node_names))], key=lambda x: x[1], reverse=True)
    print("Top affected nodes when perturbing sleep_quality:", affected[:3])


Perturbation delta (pert - orig) per node: {'sleep_quality': -0.002771735191345215, 'stress_level': 0.0033150315284729004, 'cardio_stability': -0.003277719020843506, 'pulmonary_efficiency': -0.003799617290496826, 'cognitive_load': 0.0008999109268188477}
Top affected nodes when perturbing sleep_quality: [('pulmonary_efficiency', 0.0037996173), ('stress_level', 0.0033150315), ('cardio_stability', 0.003277719)]


In [10]:
# Block 10: show directed edges (cross-corr and granger p if present) for an example snapshot
def edges_report(data: Data, node_names: List[str], top_k=10):
    """
    data.edge_attr columns: [corr_weight, lag, granger_p (or nan)]
    """
    ei = data.edge_index.cpu().numpy()
    eattr = data.edge_attr.cpu().numpy()
    edges = []
    for k in range(ei.shape[1]):
        src, dst = int(ei[0,k]), int(ei[1,k])
        w, lag, gp = eattr[k]
        edges.append((node_names[src], node_names[dst], float(w), int(lag), float(gp) if not np.isnan(gp) else None))
    edges = sorted(edges, key=lambda x: abs(x[2]), reverse=True)
    for e in edges[:top_k]:
        print(f"{e[0]} -> {e[1]} | weight={e[2]:.3f} | lag={e[3]}min | granger_p={e[4]}")

# show for the worst snapshot used above
snap_idx = worst_idx_in_test + seq_len  # predicted window index corresponding
snap_data = pyg_graphs[snap_idx]
print("Directed edges for snapshot (top):")
edges_report(snap_data, node_names, top_k=10)


Directed edges for snapshot (top):
sleep_quality -> cognitive_load | weight=-0.933 | lag=24min | granger_p=None
cognitive_load -> stress_level | weight=0.929 | lag=1min | granger_p=None
stress_level -> cognitive_load | weight=0.928 | lag=1min | granger_p=None
cognitive_load -> sleep_quality | weight=-0.927 | lag=5min | granger_p=None
sleep_quality -> stress_level | weight=-0.863 | lag=55min | granger_p=None
stress_level -> sleep_quality | weight=-0.844 | lag=1min | granger_p=None
sleep_quality -> cardio_stability | weight=0.729 | lag=2min | granger_p=None
cardio_stability -> sleep_quality | weight=0.727 | lag=8min | granger_p=None
cardio_stability -> cognitive_load | weight=-0.702 | lag=9min | granger_p=None
cognitive_load -> cardio_stability | weight=-0.693 | lag=2min | granger_p=None


In [11]:
# Block A: aggregate directed edges across all windows into a single DataFrame
import numpy as np
import pandas as pd
from collections import defaultdict

def aggregate_directed_edges(pyg_graphs, node_names, min_occurrence=1):
    """
    Collects directed edges (from each snapshot's edge_index & edge_attr),
    aggregates weight (median), lag (median), and counts occurrences.
    Returns DataFrame with columns: src, dst, count, median_weight, median_lag, granger_p_median (if present)
    """
    edge_store = defaultdict(list)  # key: (src,dst) -> list of (weight, lag, gr_p)
    for g in pyg_graphs:
        if not hasattr(g, 'edge_index') or g.edge_index is None:
            continue
        ei = g.edge_index.cpu().numpy()
        eattr = g.edge_attr.cpu().numpy() if hasattr(g, 'edge_attr') else None
        for k in range(ei.shape[1]):
            src, dst = int(ei[0,k]), int(ei[1,k])
            if eattr is None:
                weight, lag, gp = 0.0, -1, np.nan
            else:
                weight, lag, gp = float(eattr[k,0]), int(eattr[k,1]), float(eattr[k,2]) if not np.isnan(eattr[k,2]) else np.nan
            edge_store[(src,dst)].append((weight, lag, gp))
    rows = []
    for (src,dst), vals in edge_store.items():
        if len(vals) < min_occurrence:
            continue
        weights = np.array([v[0] for v in vals])
        lags = np.array([v[1] for v in vals])
        gps = np.array([v[2] for v in vals])
        rows.append({
            "src": node_names[src],
            "dst": node_names[dst],
            "src_idx": src,
            "dst_idx": dst,
            "count": len(vals),
            "median_weight": float(np.median(weights)),
            "median_lag_min": int(np.median(lags)),
            "granger_p_median": float(np.nanmedian(gps) if not np.all(np.isnan(gps)) else np.nan),
            "weights_list": weights,
            "lags_list": lags
        })
    df_edges = pd.DataFrame(rows).sort_values("median_weight", key=lambda s: s.abs(), ascending=False).reset_index(drop=True)
    return df_edges

# Example usage:
global_edges_df = aggregate_directed_edges(pyg_graphs, node_names, min_occurrence=1)
global_edges_df.head(20)


,src,dst,src_idx,dst_idx,count,median_weight,median_lag_min,granger_p_median,weights_list,lags_list
0,cognitive_load,stress_level,4,1,23,0.929256,1,NaN,"[0.9477135539054871, 0.9017395377159119, 0.952...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, ..."
1,stress_level,cognitive_load,1,4,23,0.928015,1,NaN,"[0.9487643241882324, 0.9015536308288574, 0.952...","[8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
2,sleep_quality,cognitive_load,0,4,22,-0.927903,3,NaN,"[-0.9545643925666809, -0.8698533773422241, -0....","[4, 1, 1, 1, 1, 1, 24, 2, 23, 7, 178, 18, 1, 1..."
3,cognitive_load,sleep_quality,4,0,22,-0.924469,1,NaN,"[-0.9557874798774719, -0.8696070313453674, -0....","[1, 1, 8, 4, 18, 1, 5, 1, 1, 166, 11, 1, 1, 1,..."
4,sleep_quality,stress_level,0,1,23,-0.869940,1,NaN,"[-0.9655612707138062, -0.9609084725379944, -0....","[1, 1, 1, 1, 1, 1, 55, 4, 54, 2, 55, 1, 4, 1, ..."
5,stress_level,sleep_quality,1,0,23,-0.860379,1,NaN,"[-0.9666393399238586, -0.9603537321090698, -0....","[7, 1, 1, 1, 29, 1, 1, 1, 1, 108, 52, 1, 4, 1,..."
6,cardio_stability,sleep_quality,2,0,23,0.715931,3,NaN,"[0.770922064781189, 0.5661665201187134, 0.7286...","[6, 2, 1, 1, 180, 5, 8, 1, 1, 179, 1, 2, 109, ..."
7,sleep_quality,cardio_stability,0,2,23,0.714787,3,NaN,"[0.7662836313247681, 0.5766874551773071, 0.767...","[1, 42, 82, 82, 178, 2, 2, 3, 23, 3, 3, 1, 1, ..."
8,cardio_stability,cognitive_load,2,4,23,-0.696417,9,NaN,"[-0.7451180219650269, -0.6690930724143982, -0....","[4, 1, 1, 1, 179, 1, 9, 2, 9, 1, 180, 179, 114..."
9,cognitive_load,cardio_stability,4,2,23,-0.693467,3,NaN,"[-0.7437114119529724, -0.6740134954452515, -0....","[11, 24, 140, 179, 179, 2, 2, 2, 18, 2, 180, 2..."


In [12]:
# Block B: run perturbation influence across many sequences and aggregate deltas per directed pair
import math
from tqdm import tqdm

def aggregate_perturbation_influence(model, pyg_graphs, seq_len, node_names, sample_idxs=None,
                                     perturbation=-0.3, which_snapshot=-1, topk_nodes=None):
    """
    For a set of starting indices (sample_idxs), run perturbation on each node individually and measure
    average absolute downstream delta on all other nodes. Returns DataFrame with (src, dst, mean_delta, std_delta, occurrences).
    If sample_idxs is None, uses many non-overlapping sequences (safe subset) to limit cost.
    """
    n = len(pyg_graphs)
    # build valid i such that we can take seq_len snapshots starting at i
    valid_starts = [i for i in range(0, n - seq_len)]
    if sample_idxs is None:
        # sample up to 100 sequences uniformly
        sample_idxs = np.linspace(0, len(valid_starts)-1, min(100, len(valid_starts)), dtype=int)
        sample_idxs = [valid_starts[int(x)] for x in sample_idxs]
    else:
        # ensure valid
        sample_idxs = [i for i in sample_idxs if i in valid_starts]

    influence_store = defaultdict(list)  # key (src_idx,dst_idx) -> list of deltas

    model.eval()
    for start in tqdm(sample_idxs, desc="Perturb samples"):
        seq = [pyg_graphs[j] for j in range(start, start + seq_len)]
        # original prediction
        with torch.no_grad():
            orig = model(seq).cpu().numpy()  # (num_nodes,)
        for src_idx in range(len(node_names)):
            seq_p = copy.deepcopy(seq)
            # perturb all features of src node in chosen snapshot
            seq_p[which_snapshot].x[src_idx] = seq_p[which_snapshot].x[src_idx] + perturbation
            with torch.no_grad():
                pert = model(seq_p).cpu().numpy()
            delta = pert - orig  # (num_nodes,) positive indicates increase in predicted mean
            for dst_idx in range(len(node_names)):
                influence_store[(src_idx, dst_idx)].append(float(delta[dst_idx]))

    rows = []
    for (src,dst), deltas in influence_store.items():
        arr = np.array(deltas)
        rows.append({
            "src_idx": int(src),
            "dst_idx": int(dst),
            "src": node_names[src],
            "dst": node_names[dst],
            "mean_delta": float(np.mean(arr)),
            "mean_abs_delta": float(np.mean(np.abs(arr))),
            "std_delta": float(np.std(arr)),
            "count": len(arr)
        })
    df_inf = pd.DataFrame(rows)
    # sort by mean_abs_delta descending (strongest influence)
    df_inf = df_inf.sort_values("mean_abs_delta", ascending=False).reset_index(drop=True)
    return df_inf

# Example usage (this can take time depending on sample size):
pert_inf_df = aggregate_perturbation_influence(model, pyg_graphs, seq_len, node_names, sample_idxs=None, perturbation=-0.4)
pert_inf_df.head(20)


Perturb samples: 100%|██████████| 17/17 [00:01<00:00,  8.83it/s]


,src_idx,dst_idx,src,dst,mean_delta,mean_abs_delta,std_delta,count
0,2,3,cardio_stability,pulmonary_efficiency,-0.006202,0.006202,0.000877,17
1,2,1,cardio_stability,stress_level,0.005470,0.005470,0.000745,17
2,2,2,cardio_stability,cardio_stability,-0.005285,0.005285,0.000744,17
3,3,3,pulmonary_efficiency,pulmonary_efficiency,-0.005251,0.005251,0.001675,17
4,2,0,cardio_stability,sleep_quality,-0.004757,0.004757,0.000673,17
5,3,1,pulmonary_efficiency,stress_level,0.004680,0.004680,0.001458,17
6,3,2,pulmonary_efficiency,cardio_stability,-0.004537,0.004537,0.001442,17
7,0,3,sleep_quality,pulmonary_efficiency,-0.004447,0.004447,0.000838,17
8,3,0,pulmonary_efficiency,sleep_quality,-0.004372,0.004372,0.001513,17
9,4,3,cognitive_load,pulmonary_efficiency,-0.004347,0.004347,0.000667,17


In [13]:
# Block C: merge correlation/granger evidence with perturbation evidence into a single edge score
def combine_edge_evidence(global_edges_df, pert_inf_df, node_names,
                          w_corr=0.5, w_pert=0.5, granger_boost=0.15):
    """
    Creates unified confidence per directed edge using:
      score = w_corr * normalized(|median_weight|) + w_pert * normalized(mean_abs_delta)
    If granger p-value exists and < 0.05, add granger_boost to score.
    Normalization is done across all edges to [0,1].
    Returns DataFrame with src,dst,median_lag,score and components.
    """
    # prepare perturbation lookup
    pert_map = {(r.src_idx, r.dst_idx): r for _, r in pert_inf_df.iterrows()} if not pert_inf_df.empty else {}
    # take copies
    ged = global_edges_df.copy()
    # map pert values
    pert_vals = []
    for _, row in ged.iterrows():
        key = (row.src_idx, row.dst_idx)
        if key in pert_map:
            pert_vals.append(pert_map[key]["mean_abs_delta"])
        else:
            pert_vals.append(0.0)
    ged["pert_mean_abs_delta"] = pert_vals

    # normalize median_weight absolute and pert
    def normalize_series(s):
        if s.max() == s.min():
            return np.zeros_like(s, dtype=float)
        return (s - s.min()) / (s.max() - s.min())

    ged["abs_med_weight"] = np.abs(ged["median_weight"])
    ged["w_corr_norm"] = normalize_series(ged["abs_med_weight"])
    ged["w_pert_norm"] = normalize_series(ged["pert_mean_abs_delta"])
    ged["score_base"] = w_corr * ged["w_corr_norm"] + w_pert * ged["w_pert_norm"]
    # granger boost if present and low p
    def granger_bonus(p):
        if np.isnan(p): return 0.0
        return granger_boost if p < 0.05 else 0.0
    ged["granger_bonus"] = ged["granger_p_median"].apply(granger_bonus)
    ged["score"] = ged["score_base"] + ged["granger_bonus"]
    # final normalization to [0,1]
    ged["score_norm"] = normalize_series(ged["score"])
    # sort
    ged = ged.sort_values("score_norm", ascending=False).reset_index(drop=True)
    return ged

# Example usage:
combined_edges_df = combine_edge_evidence(global_edges_df, pert_inf_df, node_names,
                                          w_corr=0.4, w_pert=0.6, granger_boost=0.15)
combined_edges_df.head(30)


,src,dst,src_idx,dst_idx,count,median_weight,median_lag_min,granger_p_median,weights_list,lags_list,pert_mean_abs_delta,abs_med_weight,w_corr_norm,w_pert_norm,score_base,granger_bonus,score,score_norm
0,cognitive_load,stress_level,4,1,23,0.929256,1,NaN,"[0.9477135539054871, 0.9017395377159119, 0.952...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, ...",0.003845,0.929256,1.000000,0.546522,0.727913,0.0,0.727913,1.000000
1,cardio_stability,stress_level,2,1,23,-0.651056,19,NaN,"[-0.7578170299530029, -0.5504528880119324, -0....","[14, 1, 1, 1, 180, 2, 47, 10, 43, 8, 180, 124,...",0.005470,0.651056,0.443636,0.859133,0.692934,0.0,0.692934,0.944028
2,sleep_quality,stress_level,0,1,23,-0.869940,1,NaN,"[-0.9655612707138062, -0.9609084725379944, -0....","[1, 1, 1, 1, 1, 1, 55, 4, 54, 2, 55, 1, 4, 1, ...",0.003947,0.869940,0.881375,0.566086,0.692201,0.0,0.692201,0.942854
3,cardio_stability,sleep_quality,2,0,23,0.715931,3,NaN,"[0.770922064781189, 0.5661665201187134, 0.7286...","[6, 2, 1, 1, 180, 5, 8, 1, 1, 179, 1, 2, 109, ...",0.004757,0.715931,0.573378,0.721941,0.662516,0.0,0.662516,0.895352
4,cognitive_load,sleep_quality,4,0,22,-0.924469,1,NaN,"[-0.9557874798774719, -0.8696070313453674, -0....","[1, 1, 8, 4, 18, 1, 5, 1, 1, 166, 11, 1, 1, 1,...",0.003249,0.924469,0.990426,0.431758,0.655225,0.0,0.655225,0.883686
5,cardio_stability,pulmonary_efficiency,2,3,17,-0.436273,6,NaN,"[-0.4590439796447754, -0.480249285697937, -0.5...","[60, 5, 2, 6, 6, 6, 6, 25, 7, 172, 128, 3, 4, ...",0.006202,0.436273,0.014100,1.000000,0.605640,0.0,0.605640,0.804341
6,stress_level,sleep_quality,1,0,23,-0.860379,1,NaN,"[-0.9666393399238586, -0.9603537321090698, -0....","[7, 1, 1, 1, 29, 1, 1, 1, 1, 108, 52, 1, 4, 1,...",0.003110,0.860379,0.862254,0.404934,0.587862,0.0,0.587862,0.775893
7,sleep_quality,cardio_stability,0,2,23,0.714787,3,NaN,"[0.7662836313247681, 0.5766874551773071, 0.767...","[1, 42, 82, 82, 178, 2, 2, 3, 23, 3, 3, 1, 1, ...",0.003795,0.714787,0.571090,0.536813,0.550524,0.0,0.550524,0.716144
8,cognitive_load,cardio_stability,4,2,23,-0.693467,3,NaN,"[-0.7437114119529724, -0.6740134954452515, -0....","[11, 24, 140, 179, 179, 2, 2, 2, 18, 2, 180, 2...",0.003738,0.693467,0.528453,0.525934,0.526941,0.0,0.526941,0.678409
9,sleep_quality,pulmonary_efficiency,0,3,17,-0.533685,5,NaN,"[-0.5521436929702759, -0.5932676792144775, -0....","[48, 6, 5, 1, 2, 2, 8, 21, 5, 3, 1, 1, 12, 3, ...",0.004447,0.533685,0.208911,0.662309,0.480950,0.0,0.480950,0.604814


In [14]:
# Block D: extract causal chains starting from a seed node using combined edge table
from heapq import heappush, heappop

def build_adj_from_combined(combined_edges_df, score_threshold=0.05):
    """
    Build adjacency dict: src_idx -> list of (dst_idx, median_lag, score_norm)
    Only include edges with score_norm >= score_threshold
    """
    adj = defaultdict(list)
    for _, r in combined_edges_df.iterrows():
        if r.score_norm < score_threshold:
            continue
        adj[int(r.src_idx)].append((int(r.dst_idx), int(r.median_lag_min), float(r.score_norm)))
    return adj

def extract_top_chains(adj, seed_idx, node_names, max_hops=4, top_k=5, min_confidence=0.05):
    """
    Greedy best-first search to extract top_k causal chains from seed node.
    Score of chain = product of (edge_score) (interpreted as joint confidence), or you can sum logs.
    Returns list of chains: each is dict {path: [names], hops, total_lag_min, chain_score, edges: [(src,dst,lag,score)...]}
    """
    # priority queue by -score so higher score first
    pq = []
    # initialize with seed
    heappush(pq, (-1.0, 0, seed_idx, [], 0))  # (neg_score, hops, current_node, path_edges, total_lag)
    results = []
    seen_paths = set()
    while pq and len(results) < top_k:
        neg_score, hops, cur, path_edges, total_lag = heappop(pq)
        score = -neg_score
        # if not initial node, generate a readable path
        if path_edges:
            path_nodes = [node_names[path_edges[0][0]]] + [node_names[e[1]] for e in path_edges]
            path_tuple = tuple(path_nodes)
            if path_tuple in seen_paths:
                continue
            seen_paths.add(path_tuple)
            results.append({
                "path": path_nodes,
                "hops": hops,
                "total_lag_min": total_lag,
                "chain_score": score,
                "edges": [{"src_idx": e[0], "dst_idx": e[1], "lag_min": e[2], "edge_score": e[3]} for e in path_edges]
            })
        if hops >= max_hops:
            continue
        # explore next edges
        for (dst, lag, escore) in adj.get(cur, []):
            new_score = score * escore  # multiplicative confidence
            if new_score < min_confidence:
                continue
            new_edges = path_edges + [(cur, dst, lag, escore)]
            heappush(pq, (-new_score, hops+1, dst, new_edges, total_lag + lag))
    return results

# Example usage:
adj = build_adj_from_combined(combined_edges_df, score_threshold=0.05)
seed = node_names.index("sleep_quality") if "sleep_quality" in node_names else 0
chains = extract_top_chains(adj, seed, node_names, max_hops=4, top_k=6, min_confidence=0.02)
for c in chains:
    print(" -> ".join(c["path"]), f"| total_lag={c['total_lag_min']}min | score={c['chain_score']:.3f}")
    for e in c["edges"]:
        print("   ", f"{node_names[e['src_idx']]} -> {node_names[e['dst_idx']]} lag={e['lag_min']} score={e['edge_score']:.3f}")
    print()


sleep_quality -> stress_level | total_lag=1min | score=0.943
    sleep_quality -> stress_level lag=1 score=0.943

sleep_quality -> stress_level -> sleep_quality | total_lag=2min | score=0.732
    sleep_quality -> stress_level lag=1 score=0.943
    stress_level -> sleep_quality lag=1 score=0.776

sleep_quality -> cardio_stability | total_lag=3min | score=0.716
    sleep_quality -> cardio_stability lag=3 score=0.716

sleep_quality -> stress_level -> sleep_quality -> stress_level | total_lag=3min | score=0.690
    sleep_quality -> stress_level lag=1 score=0.943
    stress_level -> sleep_quality lag=1 score=0.776
    sleep_quality -> stress_level lag=1 score=0.943

sleep_quality -> cardio_stability -> stress_level | total_lag=22min | score=0.676
    sleep_quality -> cardio_stability lag=3 score=0.716
    cardio_stability -> stress_level lag=19 score=0.944

sleep_quality -> cardio_stability -> sleep_quality | total_lag=6min | score=0.641
    sleep_quality -> cardio_stability lag=3 score=0.7

In [15]:
# Block E: generate human-friendly timeline narratives from extracted chains
def narrative_from_chain(chain):
    """
    chain: one element from extract_top_chains output
    returns a short human-readable timeline string.
    """
    parts = []
    cum_lag = 0
    for i, e in enumerate(chain["edges"]):
        src = node_names[e["src_idx"]]
        dst = node_names[e["dst_idx"]]
        lag = e["lag_min"]
        esc = e["edge_score"]
        cum_lag += lag
        parts.append(f"{src} → {dst} (after ~{lag} min, confidence {esc:.2f})")
    timeline = "  then  ".join(parts)
    return f"Chain (total ~{chain['total_lag_min']} min, score={chain['chain_score']:.3f}): {timeline}"

# Example: print narratives for the discovered chains
for c in chains:
    print(narrative_from_chain(c))


Chain (total ~1 min, score=0.943): sleep_quality → stress_level (after ~1 min, confidence 0.94)
Chain (total ~2 min, score=0.732): sleep_quality → stress_level (after ~1 min, confidence 0.94)  then  stress_level → sleep_quality (after ~1 min, confidence 0.78)
Chain (total ~3 min, score=0.716): sleep_quality → cardio_stability (after ~3 min, confidence 0.72)
Chain (total ~3 min, score=0.690): sleep_quality → stress_level (after ~1 min, confidence 0.94)  then  stress_level → sleep_quality (after ~1 min, confidence 0.78)  then  sleep_quality → stress_level (after ~1 min, confidence 0.94)
Chain (total ~22 min, score=0.676): sleep_quality → cardio_stability (after ~3 min, confidence 0.72)  then  cardio_stability → stress_level (after ~19 min, confidence 0.94)
Chain (total ~6 min, score=0.641): sleep_quality → cardio_stability (after ~3 min, confidence 0.72)  then  cardio_stability → sleep_quality (after ~3 min, confidence 0.90)
